In [1]:
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM, GRU, TimeDistributedDense
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from keras import backend as K
import kagglegym
import time

Using Theano backend.


In [ ]:
import kagglegym
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

env = kagglegym.make()
o = env.reset()
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = ['technical_30', 'technical_20', 'fundamental_11', 'technical_19']
#[c for c in o.train.columns if c not in excl]

train = pd.read_hdf('../input/train.h5')
train = train[col]
d_mean= train.median(axis=0)

train = o.train[col]
n = train.isnull().sum(axis=1)
for c in train.columns:
    train[c + '_nan_'] = pd.isnull(train[c])
    d_mean[c + '_nan_'] = 0
    
train = train.fillna(d_mean)

new_data = np.zeros((0, 8))
for data_id in np.unique(o.train.id):
    data_filter = (o.train.id == data_id)
    data_filtered = data[data_filter].values
    timestamps_filtered = o.train[data_filter].timestamp.values
    data_prime = np.subtract(data_filtered, np.concatenate(([list(d_mean)], data_filtered.values), axis=0)[:-1])
    new_data_for_id = pd.DataFrame(np.concatenate((data_filtered, data_prime), axis=1), columns=col + [column + '_prime_' for column in col])
    new_data = np.concatenate((new_data, new_data_for_id), axis=0)

columns = [train[c + '_past_'] for c in train.columns]

train['znull'] = n
n = []

rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
model1 = rfr.fit(train, o.train['y'])

#https://www.kaggle.com/bguberfain/two-sigma-financial-modeling/univariate-model-with-clip/run/482189
low_y_cut = -0.075
high_y_cut = 0.075
y_is_above_cut = (o.train.y > high_y_cut)
y_is_below_cut = (o.train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
model2 = LinearRegression(n_jobs=-1)
model2.fit(np.array(o.train[col].fillna(d_mean).loc[y_is_within_cut, 'technical_20'].values).reshape(-1,1), o.train.loc[y_is_within_cut, 'y'])
train = []

#https://www.kaggle.com/ymcdull/two-sigma-financial-modeling/ridge-lb-0-0100659
ymean_dict = dict(o.train.groupby(["id"])["y"].median())

while True:
    test = o.features[col]
    n = test.isnull().sum(axis=1)
    for c in test.columns:
        test[c + '_nan_'] = pd.isnull(test[c])
    test = test.fillna(d_mean)
    test['znull'] = n
    pred = o.target
    test2 = np.array(o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
    pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * 0.65) + (model2.predict(test2).clip(low_y_cut, high_y_cut) * 0.35)
    pred['y'] = pred.apply(lambda r: 0.95 * r['y'] + 0.05 * ymean_dict[r['id']] if r['id'] in ymean_dict else r['y'], axis = 1)
    pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
    o, reward, done, info = env.step(pred)
    if done:
        print("el fin ...", info["public_score"])
        break
    if o.features.timestamp[0] % 100 == 0:
        print(reward)

In [45]:
data = pd.read_hdf('data/train.h5')
train_bounds = (0, 200)
test_bounds = (200, 300) # 1812
data = data[columns + ['id', 'timestamp', 'y']]

In [47]:
train = data[(data.id > train_bounds[0]) & (data.id < train_bounds[1])]
test = data[(data.id > test_bounds[0]) & (data.id < test_bounds[1])]
means = train.mean()
train = train.fillna(means)
test = test.fillna(means)
data = None

In [62]:
model = Sequential()
model.add(LSTM(128, batch_input_shape=[32, 32, 32], return_sequences=True, stateful=True))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False, stateful=True))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))

model.compile(loss='mse',
              optimizer='adam',
              metrics=[])

In [69]:
model.updates

[(<TensorType(float32, matrix)>, DimShuffle{0,1}.0),
 (<TensorType(float32, matrix)>, DimShuffle{0,1}.0),
 (batchnormalization_4_running_mean, Elemwise{add,no_inplace}.0),
 (batchnormalization_4_running_std, Elemwise{add,no_inplace}.0),
 (<TensorType(float32, matrix)>, DimShuffle{0,1}.0),
 (<TensorType(float32, matrix)>, DimShuffle{0,1}.0),
 (batchnormalization_5_running_mean, Elemwise{add,no_inplace}.0),
 (batchnormalization_5_running_std, Elemwise{add,no_inplace}.0)]

In [67]:
model.state_updates

[]

In [66]:
model.reset_states()

In [68]:
model.stateful

True

In [48]:
%%time
clf = RandomForestRegressor()
clf.fit(train.drop(['id', 'timestamp', 'y'], axis=1).values, train['y'].values)

CPU times: user 10min 41s, sys: 5.05 s, total: 10min 46s
Wall time: 11min 11s


In [49]:
clf.score(test.drop(['id', 'timestamp', 'y'], axis=1).values, test['y'].values)

-0.27078327175892936

In [56]:
from sklearn.metrics import r2_score
def r_score(y_true, y_pred, sample_weight=None, multioutput=None):
    r2 = r2_score(y_true, y_pred, sample_weight=sample_weight,
                  multioutput=multioutput)
    r = (np.sign(r2)*np.sqrt(np.abs(r2)))
    if r <= -1:
        return -1
    else:
        return r

In [57]:
r_score(test['y'].values, clf.predict(test.drop(['id', 'timestamp', 'y'], axis=1).values))

-0.52036840003878926

In [50]:
a = [(name, importance) for name, importance in zip(train.columns.drop(['id', 'timestamp', 'y']), clf.feature_importances_)]

In [51]:
columns = [name for name, importance in sorted(a, key=lambda row: -row[1])[:50]]

In [52]:
columns

['technical_33',
 'technical_41',
 'technical_24',
 'technical_3',
 'technical_1',
 'technical_30',
 'technical_5',
 'technical_25',
 'technical_31',
 'technical_44',
 'technical_28',
 'technical_20',
 'technical_21',
 'technical_13',
 'technical_19',
 'technical_27',
 'technical_36',
 'technical_35',
 'technical_40',
 'fundamental_41',
 'fundamental_53',
 'fundamental_42',
 'fundamental_0',
 'fundamental_10',
 'fundamental_62',
 'derived_3',
 'fundamental_7',
 'fundamental_36',
 'technical_7',
 'fundamental_45',
 'technical_17',
 'fundamental_21',
 'fundamental_18',
 'technical_2',
 'fundamental_33',
 'technical_11',
 'derived_1',
 'fundamental_23',
 'fundamental_48',
 'fundamental_15',
 'derived_0',
 'fundamental_44',
 'fundamental_30',
 'fundamental_29',
 'fundamental_59',
 'fundamental_39',
 'fundamental_34',
 'fundamental_32',
 'fundamental_57',
 'fundamental_60']

In [44]:
columns

['technical_33',
 'technical_41',
 'technical_24',
 'technical_30',
 'technical_3',
 'technical_1',
 'technical_31',
 'technical_28',
 'technical_5',
 'technical_44',
 'technical_25',
 'technical_20',
 'technical_13',
 'technical_21',
 'technical_19',
 'technical_27',
 'technical_36',
 'technical_35',
 'fundamental_62',
 'fundamental_41',
 'fundamental_10',
 'technical_40',
 'technical_17',
 'fundamental_42',
 'fundamental_53',
 'derived_3',
 'fundamental_0',
 'technical_2',
 'fundamental_7',
 'technical_7',
 'fundamental_36',
 'fundamental_18',
 'fundamental_21',
 'fundamental_33',
 'technical_11',
 'derived_1',
 'fundamental_48',
 'fundamental_30',
 'fundamental_29',
 'derived_0',
 'fundamental_60',
 'fundamental_23',
 'fundamental_45',
 'fundamental_44',
 'fundamental_59',
 'fundamental_15',
 'fundamental_57',
 'fundamental_34',
 'fundamental_39',
 'fundamental_32']

In [ ]:
'technical_30', 'technical_20', 'fundamental_11', 'technical_19'